In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, ConvBertForSequenceClassification, ConvBertTokenizer
from transformers import AutoTokenizer

# Load dataset
# df = pd.read_csv('datasets/cleaned_OLID_test.tsv', sep="\t")
# df = pd.read_csv('datasets/cleaned_OLID.tsv', sep="\t")
# df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep='\t')
# df = pd.read_csv('datasets/cleaned_tr_offenseval_test.tsv', sep='\t')
# df = pd.read_csv('datasets/cleaned_hatespeech_offensive.tsv', sep='\t')
# df = pd.read_csv('json_samples/wolf.csv')
# df = pd.read_csv('json_samples/PulpFiction.tsv', sep='\t')
# df = pd.read_csv('json_samples/WolfOfWallstreet.tsv', sep='\t')
# Assuming your columns are named 'tweet' and 'class', change accordingly
# tweets = df['tweet'].values
# labels_df = df['label'].values
# tweets = df['text']
tweets = ['Jews and monkeys have so much in common', 'Jews would love those dogs']
df = tweets
# Split the dataset into training and validation sets
# _, tweets, _, labels_df = train_test_split(tweets, labels_df, test_size=0.2, random_state=42)

# val_texts = tweets
# Initialize BERT tokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = ConvBertTokenizer.from_pretrained('YituTech/conv-bert-base')
# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") ## bu ve alttaki değişiyor, birde buna göre importlar değişir

# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

# Tokenize and encode the training and validation texts
# train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
# encodings = tokenizer(tweets.tolist(), truncation=True, padding=True)
encodings = tokenizer(tweets, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        # self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return df.__len__()

# train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(encodings)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=12, shuffle=False)


In [8]:
import torch
from torch import optim
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
# from transformers import AutoTokenizer, RobertaForSequenceClassification
import time
# Initialize BERT model for sequence classification
# model = RobertaForSequenceClassification.from_pretrained('models/SOLID_semi_RoBERTa_2_cleaned')
# model = BertForSequenceClassification.from_pretrained('models/troffenseval_SOLIDBERT')
model = ConvBertForSequenceClassification.from_pretrained('models/SOLID_ConvBERT_1', num_labels=2)
# model = DistilBertForSequenceClassification.from_pretrained("models/OLIDdistilBERT_1")

# Define optimizer and learning rate
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


ConvBertForSequenceClassification(
  (convbert): ConvBertModel(
    (embeddings): ConvBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ConvBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x ConvBertLayer(
          (attention): ConvBertAttention(
            (self): ConvBertSelfAttention(
              (query): Linear(in_features=768, out_features=384, bias=True)
              (key): Linear(in_features=768, out_features=384, bias=True)
              (value): Linear(in_features=768, out_features=384, bias=True)
              (key_conv_attn_layer): SeparableConv1D(
                (depthwise): Conv1d(768, 768, kernel_size=(9,), stride=(1,), padding=(4,), groups=768, bias=False)
                (pointwise): Conv1d(7

In [9]:
import numpy as np
import time
# Evaluation
model.eval()
correct = 0
total = 0
i = 0
prediction_list = np.array([])

with torch.no_grad():
    test_start  = time.time()
    print('start')
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        prediction_list = np.append(prediction_list, predictions.detach().cpu().numpy())
    print('end')
    
    test_end = time.time()



start
end


In [12]:
prediction_list

array([0., 0.])

In [10]:
# import pandas as pd

# df_original = pd.read_csv('json_samples/WolfOfWallstreet.tsv', sep='\t')


# df_out = df_original.assign(predictions=prediction_list)


# df_out.to_csv('outputs/OUTPUTwolfofwallstreet.tsv',index=False, sep='\t')

In [11]:
test_time = test_end - test_start

print(f'test time: {test_time}')

test time: 1.8804938793182373
